In [2]:
import msnoise.api as ms
import numpy as np
import matplotlib.pyplot as plt

In [6]:
path_ini = 'D:\\MEMOIRE\\MSNoise\\ablation\\db.ini'
database = ms.connect(inifile=path_ini)

In [7]:
ms.read_db_inifile(inifile=path_ini)

IniFile(tech=2, hostname='127.0.0.1', database='ablation', username='msnoise', password='JulGov3!', prefix='')

In [17]:
filt = ms.get_stations(database)